In [1]:
import solution_file_processing as sfp
import os
import numpy as np
import pandas as pd
from dask import dataframe as dd

from solution_file_processing.utils.utils import catch_errors
from solution_file_processing.utils.write_excel import write_xlsx_column, write_xlsx_stack, STACK_PALETTE, IEA_PALETTE_16, IEA_PALETTE_PLUS, EXTENDED_PALETTE
from solution_file_processing.constants import VRE_TECHS
from solution_file_processing.timeseries import create_output_11 as create_ts_output_11
from solution_file_processing.timeseries import create_output_4 as create_timeseries_output_4
from solution_file_processing import log
from solution_file_processing.plots import _get_plot_1_variables

# Initialize config with toml file
c = sfp.SolutionFilesConfig('config_archive/ukraine/UKR.toml')

from solution_file_processing.plots import create_plot_1a

[10:23:24 00:00] I:117:__init__ - Logging to C:\Users\hart_c\showcase\solution-file-processing\logs\20240425_102324-UKR.log.
[10:23:24 00:00] I:149:__init__ - Initialized SolutionFilesConfig for config_archive/ukraine/UKR.toml.


In [2]:
# would be good to make palettes etc based on the regions for consistency purposes
reg_ids = list(set(c.v.load_by_reg.reset_index()[c.GEO_COLS[0]].values))
reg_palette = {reg_ids[i]: IEA_PALETTE_16[i] for i in range(len(reg_ids))}

# Model palette
model_ids = list(set(c.v.load_by_reg.reset_index()['model'].values))
# Use extended palette so it can have more than 16 variables
model_palette = {model_ids[i]: IEA_PALETTE_16[i] for i in range(len(model_ids))}

# Regions and technologies will always be consistent this way. May need to be copied to other parts of the code
combined_palette = dict(STACK_PALETTE, **reg_palette, **model_palette)

[10:23:24 00:00] I:47:_drive_cache_wrapper - Loading from objects cache: node_yr_df.parquet (pd.DataFrame).


In [3]:
plot_cols = {
        "load_by_reg": c.v.customer_load_by_reg.groupby(["model", c.GEO_COLS[0]]).sum().value.unstack(level="model") / 1000,
        "pk_load_by_reg": c.v.customer_load_reg_ts.groupby(c.GEO_COLS[0], axis=1).sum().stack(c.GEO_COLS[0]).groupby(
            ["model", c.GEO_COLS[0]]).max().unstack(level="model") / 1000,
        "pk_netload_by_reg": c.v.net_load_reg_ts.groupby(c.GEO_COLS[0], axis=1)
                             .sum()
                             .stack(c.GEO_COLS[0])
                             .groupby(["model", c.GEO_COLS[0]])
                             .max()
                             .unstack(level="model")
                             / 1000,
        "line_cap_reg": c.v.line_cap_reg.value.unstack(level="line")/ 1000,
        "line_net_exports_reg": ( c.v.line_imp_exp_reg["Flow"] - c.v.line_imp_exp_reg["Flow Back"]).unstack("line")/ 1000,
        "line_exports_reg": (c.v.line_imp_exp_reg["Flow"]).unstack("line") / 1000,
        "line_imports_reg": (c.v.line_imp_exp_reg["Flow Back"]).unstack("line") / 1000,
        #              'use_by_reg': use_by_reg.groupby(['model','Region']).sum().unstack(level='Region'),
        "use_by_reg": c.v.use_by_reg.groupby(level=['model', c.GEO_COLS[0]]) \
                        .sum() \
                        .value \
                        .unstack(c.GEO_COLS[0])/ 1000,
        "gen_by_tech": c.v.gen_by_tech_reg.stack(c.GEO_COLS)
                       .groupby(["model", "Category"])
                       .sum()
                       .unstack(level="Category")
                       / 1000,
        "gen_by_reg": c.v.gen_by_tech_reg.stack(c.GEO_COLS)
                      .groupby(["model", c.GEO_COLS[0]])
                      .sum()
                      .unstack(level=c.GEO_COLS[0])
                      / 1000,
        "net_gen_by_reg": c.v.gen_by_tech_reg.stack(c.GEO_COLS)
                          .groupby(["model", c.GEO_COLS[0]])
                          .sum()
                          .unstack(level=c.GEO_COLS[0])
                          .fillna(0)
                          / 1000
                          - c.v.load_by_reg.groupby(["model", c.GEO_COLS[0]]).sum().value.unstack(level=c.GEO_COLS[0]) / 1000,
        "gen_cap_by_reg": c.v.gen_cap_tech_reg.stack(c.GEO_COLS)
                          .groupby(["model", c.GEO_COLS[0]])
                          .sum()
                          .unstack(level=c.GEO_COLS[0])
                          / 1000,
        "gen_cap_by_tech": c.v.gen_cap_tech_reg.stack(c.GEO_COLS)
                           .groupby(["model", "Category"])
                           .sum()
                           .unstack(level="Category")
                           / 1000,
        "cf_tech": c.v.cf_tech,
        "cf_tech_transposed": c.v.cf_tech.T,
        "vre_by_reg_byGen": pd.concat([c.v.vre_by_reg, c.v.vre_share.rename('Overall')], axis=1),
        "vre_by_reg_byAv": pd.concat([c.v.vre_av_by_reg, c.v.vre_av_share.rename('Overall')], axis=1),
        "re_by_reg": pd.concat([c.v.re_by_reg, c.v.vre_share.rename('Overall')],axis=1),
        "curtailment_rate": c.v.curtailment_rate / 100,
        "re_curtailed_by_tech": c.v.re_curtailment_rate_by_tech,
        ### fuels by type shouldnt be 
        "fuels_by_type": c.v.fuel_by_type.groupby(["model", "Type"])
                            .sum()
                            .value
                            .unstack(level="Type")
                            .fillna(0),
        #              'fuels_by_subtype': fuel_by_type.groupby(['model', 'Category']).sum().unstack('Category').replace(0,np.nan).dropna(axis=1,how="all").fillna(0),
        "co2_by_tech": c.v.co2_by_tech_reg.groupby(["model", "Category"])
                       .sum()
                       .value
                       .unstack(level="Category")
                       / 1e6,
        "co2_by_fuels": c.v.co2_by_fuel_reg.groupby(["model", "Type"])
                        .sum()
                        .value
                        .unstack("Type")
                        / 1e6,
        "co2_by_reg": c.v.co2_by_tech_reg.groupby(["model", c.GEO_COLS[0]])
                      .sum()
                      .value
                      .unstack(level=c.GEO_COLS[0])
                      / 1e6,
        "co2_intensity_reg": c.v.co2_by_reg.unstack(c.GEO_COLS).groupby(c.GEO_COLS[0], axis=1).sum()
                             / c.v.gen_by_tech_reg.groupby("model").sum().groupby(c.GEO_COLS[0], axis=1).sum(),

        "op_costs_by_prop": c.v.gen_op_costs_by_reg.groupby(["model", "property"]) \
                                    .sum()
                                    .unstack(level="property"),
        #'lcoe_by_tech' : lcoe_tech.unstack(level='Category'),
        #              'lcoe_by_tech_T' : lcoe_tech.unstack(level='model'),
        "ramp_pc_by_reg": pd.concat(
            [
                c.v.ramp_reg_pc_ts.groupby(['model',c.GEO_COLS[0]]).max().value.unstack(c.GEO_COLS[0]),
                c.v.ramp_pc_ts.groupby(["model"]).max().rename("Overall"),
            ],
            axis=1,
            ),
        "th_ramp_pc_by_reg": pd.concat(
            [
                c.v.th_ramp_reg_pc_ts.groupby(['model',c.GEO_COLS[0]]).max().value.unstack(c.GEO_COLS[0]),
                c.v.th_ramp_pc_ts.groupby(["model"]).max().rename("Overall")
            ],
            axis=1,
            ),

        "ramp_by_reg": pd.concat(
            [
                c.v.ramp_reg_ts.unstack(c.GEO_COLS)
                .groupby(level=c.GEO_COLS[0], axis=1)
                .sum()
                .groupby(["model"])
                .max(),
                c.v.ramp_ts.groupby(["model"]).max().value.rename("Overall"),
            ],
            axis=1,
        ),
        "th_ramp_by_reg": pd.concat(
            [
                c.v.th_ramp_reg_ts.unstack(c.GEO_COLS)
                .groupby(level=c.GEO_COLS[0], axis=1)
                .sum()
                .groupby(["model"])
                .max(),
                c.v.th_ramp_ts.groupby(["model"]).max().value.rename("Overall"),
            ],
            axis=1,
        )
}

[10:23:24 00:00] I:44:_drive_cache_wrapper - Loading from objects cache: node_df.parquet (dd.DataFrame).
[10:23:29 00:04] I:44:_drive_cache_wrapper - Loading from objects cache: gen_df.parquet (dd.DataFrame).


KeyboardInterrupt: 

In [4]:
(c.v.fuel_by_type.groupby(["model", c.GEO_COLS[0], "Type"])
                            .sum()
                            .value
                            .unstack(level="Type")
                            .fillna(0)
)

[10:24:22 00:53] I:47:_drive_cache_wrapper - Loading from objects cache: fuel_yr_df.parquet (pd.DataFrame).


Type                                                  Biofuel          Coal  \
model                                      Region                             
UKR_2021_Validation                        CR      514.513163  72798.423127   
                                           CRM       0.000000      0.000000   
                                           EOR       0.000000  37500.001068   
                                           ER        0.000000   7814.006208   
                                           NR        0.000000  34524.954362   
...                                                       ...           ...   
UKR_2030_Load-BR_2021_Cap_ST_LT_SolGas_CO2 ER        0.000000  12726.531368   
                                           NR        0.000000  96591.722820   
                                           SR        0.000000      0.000000   
                                           SWR       0.000000  30103.359514   
                                           WR        0.000000  59189.722056   

Type                                                        Gas   H2  NH3  \
model                                      Region                           
UKR_2021_Validation                        CR      55226.725400  0.0  0.0   
                                           CRM         0.000000  0.0  0.0   
                                           EOR     25146.223290  0.0  0.0   
                                           ER       5753.183414  0.0  0.0   
                                           NR      47774.840484  0.0  0.0   
...                                                         ...  ...  ...   
UKR_2030_Load-BR_2021_Cap_ST_LT_SolGas_CO2 ER       5775.504392  0.0  0.0   
                                           NR      50049.809706  0.0  0.0   
                                           SR       8523.643493  0.0  0.0   
                                           SWR       551.730597  0.0  0.0   
                                           WR       9852.211227  0.0  0.0   

Type                                                      Oil        Uranium  
model                                      Region                             
UKR_2021_Validation                        CR      4922.19072  878504.973936  
                                           CRM        0.00000       0.000000  
                                           EOR      448.53108       0.000000  
                                           ER         0.00000       0.000000  
                                           NR         0.00000       0.000000  
...                                                       ...            ...  
UKR_2030_Load-BR_2021_Cap_ST_LT_SolGas_CO2 ER         0.00000       0.000000  
                                           NR         0.00000       0.000000  
                                           SR         0.00000       0.000000  
                                           SWR        0.00000       0.000000  
                                           WR      1613.41560       0.000000  

[72 rows x 7 columns]

In [3]:
df=  c.v.customer_load_by_reg.groupby(["model", c.GEO_COLS[0]]).sum().value.unstack(level="model") / 1000

In [17]:
fig_path = './test_xlsx_write.xlsx'
with pd.ExcelWriter(fig_path, engine="xlsxwriter") as writer: # pylint: disable=abstract-class-instantiated    
    write_xlsx_column(
        df=df,
        writer=writer,
        sheet_name='test',
        subtype='clustered',
        units='NA',
        palette=combined_palette
    )

In [12]:
palette = combined_palette
col_num=2
fill_colour = palette[df.columns[col_num - df.index.nlevels]]

In [15]:
model_palette

{'UKR_2025_BAU_DamagedTx': ['rl',
  'ol',
  'gl',
  'bl',
  'pl',
  'grey10',
  'yl',
  'tl',
  'r',
  'o',
  'y',
  'g',
  't',
  'b',
  'p',
  'grey50'],
 'UKR_2021_Validation': ['rl',
  'ol',
  'gl',
  'bl',
  'pl',
  'grey10',
  'yl',
  'tl',
  'r',
  'o',
  'y',
  'g',
  't',
  'b',
  'p',
  'grey50'],
 'UKR_2030_IR_Load-BAU': ['rl',
  'ol',
  'gl',
  'bl',
  'pl',
  'grey10',
  'yl',
  'tl',
  'r',
  'o',
  'y',
  'g',
  't',
  'b',
  'p',
  'grey50'],
 'UKR_2023_Base_DamagedTx': ['rl',
  'ol',
  'gl',
  'bl',
  'pl',
  'grey10',
  'yl',
  'tl',
  'r',
  'o',
  'y',
  'g',
  't',
  'b',
  'p',
  'grey50'],
 'UKR_2030_IR_Load-BR': ['rl',
  'ol',
  'gl',
  'bl',
  'pl',
  'grey10',
  'yl',
  'tl',
  'r',
  'o',
  'y',
  'g',
  't',
  'b',
  'p',
  'grey50'],
 'UKR_2030_IR_Load-BR_ST_LT_SolGas_CO2': ['rl',
  'ol',
  'gl',
  'bl',
  'pl',
  'grey10',
  'yl',
  'tl',
  'r',
  'o',
  'y',
  'g',
  't',
  'b',
  'p',
  'grey50']}

In [ ]:
IEA_PALETTE_PLUS